In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import gc
import warnings

warnings.filterwarnings('ignore')

In [2]:
def get_psi(train,test,f_cols):
    psi_res = pd.DataFrame()
    psi_dict={}
    for c in tqdm(f_cols):
        try:
            t_train = train[c].fillna(-998)
            t_test = test[c].fillna(-998)
            #获取切分点
            bins=[]
            for i in np.arange(0,1.1,0.2):
                bins.append(t_train.quantile(i))
            bins=sorted(set(bins))
            bins[0]=-np.inf
            bins[-1]=np.inf
            #计算psi
            t_psi = pd.DataFrame()
            t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
            t_psi['test'] = pd.cut(t_test,bins).value_counts()
            if c == 'outdoorTemp':
                print(t_psi['train'])
                print(t_psi['test'])
            t_psi.index=[str(x) for x in t_psi.index]
            t_psi.loc['总计',:] = t_psi.sum()
            t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
            t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
            t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
            t_psi.loc['总计','psi'] = t_psi['psi'].sum()
            t_psi.index.name=c
            #汇总
            t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                                 columns=['变量名','PSI'])
            psi_res = pd.concat([psi_res,t_res])
            psi_dict[c]=t_psi
            print(c,'done')
        except:
            print(c,'error')
    return psi_res,psi_dict

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.DataFrame(test['time'])

In [4]:
train = train[train['temperature'].notnull()]
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')

In [5]:
train.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                 'indoorHum', 'indoorAtmo', 'temperature']
test.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                'indoorHum', 'indoorAtmo']

In [6]:
data = pd.concat([train, test])

In [7]:
data['indoorAtmo-outdoorAtmo'] = data['indoorAtmo'] - data['outdoorAtmo']
data['indoorHum-outdoorHum'] = data['indoorHum'] - data['outdoorHum']

In [8]:
group_feats = []
for i in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']):
    data['MDH_{}_mean'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('mean')
    data['MDH_{}_median'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('median')
    data['MDH_{}_max'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('max')
    data['MDH_{}_min'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('min')
    data['MDH_{}_std'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('std')
    data['MDH_{}_skew'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('skew')
    
    group_feats.append('MDH_{}_mean'.format(i))
    group_feats.append('MDH_{}_median'.format(i))

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:32<00:00,  4.57s/it]


In [9]:
# 基本交叉特征
for f1 in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum'] + group_feats):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum'] + group_feats:
        if f1 != f2:
            colname = '{}_{}_ratio'.format(f1, f2)
            data[colname] = data[f1].values / data[f2].values

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 33.18it/s]


In [10]:
data.fillna(method='bfill', inplace=True)

In [11]:
# 历史信息提取
# dt 14-44
data['dt'] = data['day'].values + (data['month'].values - 3) * 31

for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum', 'temperature']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data[data['dt'] < t].groupby(['hour'])[f].agg({'mean'}).reset_index()
        tmp.columns = ['hour', 'hit_{}_mean'.format(f)]
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)

    data = data.merge(tmp_df, on=['dt', 'hour'], how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 17.73it/s]


In [12]:
data.fillna(method='bfill', inplace=True)

In [13]:
# 离散化
for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']:
    data[f + '_20_bin'] = pd.cut(data[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_50_bin'] = pd.cut(data[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_100_bin'] = pd.cut(data[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_200_bin'] = pd.cut(data[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [14]:
for f1 in tqdm(['outdoorTemp_20_bin', 'outdoorHum_20_bin', 'outdoorAtmo_20_bin', 'indoorHum_20_bin', 'indoorAtmo_20_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.47s/it]


In [ ]:
for f1 in tqdm(['outdoorTemp_50_bin', 'outdoorHum_50_bin', 'outdoorAtmo_50_bin', 'indoorHum_50_bin', 'indoorAtmo_50_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:17<00:04,  4.25s/it]

In [ ]:
for f1 in tqdm(['outdoorTemp_100_bin', 'outdoorHum_100_bin', 'outdoorAtmo_100_bin', 'indoorHum_100_bin', 'indoorAtmo_100_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
for f1 in tqdm(['outdoorTemp_200_bin', 'outdoorHum_200_bin', 'outdoorAtmo_200_bin', 'indoorHum_200_bin', 'indoorAtmo_200_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
drop_columns = ["time", "year", "sec", "temperature"]

train_count = train.shape[0]
train = data[:train_count].copy().reset_index(drop=True)
test = data[train_count:].copy().reset_index(drop=True)
del data
gc.collect()

In [ ]:
features = train[:1].drop(drop_columns, axis=1).columns
x_train = train[features]
x_test = test[features]

y_train = train['temperature'].values - train['outdoorTemp'].values

In [ ]:
psi_res, psi_dict = get_psi(x_train,x_test,features)
features = list(psi_res[psi_res['PSI'] <= 0.2]['变量名'].values) + ['outdoorTemp']

In [ ]:
x_train = train[features]
x_test = test[features]

In [ ]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)
params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.001,
    'seed': 2020,
    'nthread': 36,
    'silent': True,
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model = xgb.train(params,
                  train_matrix,
                  num_boost_round=50000,
                  evals=watchlist,
                  verbose_eval=500,
                  early_stopping_rounds=1000)
val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)
test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

In [ ]:
sub["temperature"] = test_pred[:, 0] + test['outdoorTemp'].values
sub.to_csv('../sub/sub_baseline_psi.csv', index=False)